In [1]:
from datetime import datetime
print(f'Päivitetty {datetime.now()}')

Päivitetty 2022-10-25 17:15:09.187213


# Pika-analyysi Exceliin

Frekvenssitaulukot, ristiintaulukoinnit, tilastolliset tunnusluvut ja korrelaatiot Exceliin.

Pika-analyysiä voit käyttää omaan dataan vaihtamalla avattavan datan tilalle oma data.

Koodin suorittamisen jälkeen tulokset avautuvat Exceliin.

Voit tarvittaessa muuttaa RAJA-vakion arvoa.

In [2]:
import pandas as pd
import xlwings as xw

RAJA = 10 # jos ainutkertaisia arvoja enemmän, niin muuttujaa pidetään määrällisenä

# Vaihda tähän oma data
df = pd.read_excel('https://taanila.fi/data1.xlsx')

In [3]:
# Excel-työkirja (workbook)
app = xw.App(visible=False)
wb = xw.books.active

# Määrällisten ja kategoristen muuttujien erottelu
kvantit = []
kategoriset = []
for var in df:
    if len(df[var].unique()) > RAJA:
        kvantit.append(var)
    else:
        kategoriset.append(var)


# Korrelaatiot                
                
# Korrelaatiot Korrelaatiot-taulukkovälilehdelle
if kvantit:
    ws4 = wb.sheets.add('Korrelaatiot')
    df1 = df[kvantit].corr(min_periods=10)
    ws4.range(1, 1).value = \
        'Määrällisten ja mielipideasteikollisten muuttujien korrelaatiot (n>9)'
    ws4.range(1, 1).font.bold = True
    ws4.range(3, 1).value = df1

    # Korrelaatiot kategoristen muuttujien määräämissä ryhmissä
    if kategoriset:
        rivi = df1.shape[0]+6
        ws4.range(rivi, 1).value = \
            'Korrelaatiot kategoristen muuttujien määräämissä ryhmissä (n>9)'
        ws4.range(rivi, 1).font.bold = True
        for var in kategoriset:
            df1 = df.groupby(var)[kvantit].corr(min_periods=10)
            ws4.range(rivi+2, 1).value = df1
            rivi = rivi + df1.shape[0]+2

            
# Tunnusluvut
            
# Tunnusluvut määrällisille ja mielipideasteikollisille Tunnusluvut-taulukkovälilehdelle
if kvantit:
    ws3 = wb.sheets.add('Tunnusluvut')
    ws3.range(1, 1).value = 'Määrällisten ja mielipideasteikollisten muuttujien tunnusluvut'
    ws3.range(1, 1).font.bold = True
    df1 = df[kvantit].describe()
    ws3.range(3, 1).value = df1
    ws3.range(4, 1).value = 'n'
    ws3.range(5, 1).value = 'keskiarvo'
    ws3.range(6, 1).value = 'keskihajonta'
    ws3.range(7, 1).value = 'pienin'
    ws3.range(11, 1).value = 'suurin'

    # Tunnusluvut kategoristen määräämissä ryhmissä
    if kategoriset:
        ws3.range(14, 1).value = 'Tunnusluvut kategoristen muuttujien määräämissä ryhmissä'
        ws3.range(14, 1).font.bold = True
        rivi = df1.shape[0] + 8
        for var1 in kategoriset:
            for var2 in kvantit:
                if var1 != var2:
                    df1 = df.groupby(var1)[var2].describe()
                    df1.index.name = var1
                    ws3.range(rivi, 2).value = var2
                    ws3.range(rivi+1, 1).value = df1
                    ws3.range(rivi+1, 2).value = 'n'
                    ws3.range(rivi+1, 3).value = 'keskiarvo'
                    ws3.range(rivi+1, 4).value = 'keskihajonta'
                    ws3.range(rivi+1, 5).value = 'pienin'
                    ws3.range(rivi+1, 9).value = 'suurin'
                    rivi = rivi + df1.shape[0] + 3

    
# Ristiintaulukoinnit

if len(kategoriset) > 1:
    rivi = 4
    ws2 = wb.sheets.add('Ristiintaulukoinnit')
    for var1 in kategoriset:
        for var2 in kategoriset:
            if var1 != var2:
                df1 = pd.crosstab(df[var1], df[var2], margins=True)
                df2 = pd.crosstab(df[var1], df[var2], normalize='columns')
                df2.loc['n'] = df1.loc['All']
                ws2.range(rivi, 2).value = var2
                ws2.range(rivi, df1.shape[1] + 4).value = var2
                ws2.range(rivi+1, 1).value = df1
                ws2.range(rivi+1, df1.shape[1]+1).value = 'yhteensä'
                ws2.range(rivi+df1.shape[0]+1, 1).value = 'yhteensä'
                ws2.range((rivi+1, df1.shape[1]+3)).value = df2
                ws2.range((rivi+2, df1.shape[1]+4),(rivi+df2.shape[0], 
                    df1.shape[1]+3+df2.shape[1])).number_format = '0,0 %'
                rivi = rivi + df2.shape[0] + 3
    ws2.range(1, 1).value = 'Ristiintaulukoinnit kategorisille muuttujille'
    ws2.range(2, 1).value = \
        'Prosentit ovat prosentteja sarakkeen kokonaismäärästä (n)'
    ws2.range((1, 1), (2, 1)).font.bold = True
 

# Frekvenssitaulukot

if kategoriset:
    # Pidän kirjaa Excelin rivinumerosta rivi-muuttujan avulla
    rivi = 3
    ws1 = wb.sheets.add('Frekvenssitaulukot')
    # Käyn for-silmukalla läpi kaikki kategoriset muuttujat
    for var in kategoriset:
        # Lasken frekvenssit df1-nimiseen dataframeen
        df1 = pd.crosstab(df[var], 'f')
        # Lasken df1:een prosentit
        df1['%'] = df1/df1.sum()
        # Lisään df1:een Yhteensä-rivin
        df1.loc['yhteensä'] = df1.sum()
        # Frekvenssitaulukko Exceliin
        ws1.range((rivi, 1)).value = df1
        ws1.range((rivi+1, 3),(rivi+len(df1), 3)).number_format = '0,0 %'
        ws1.range((rivi, 2), (rivi, 3)).api.HorizontalAlignment = \
            xw.constants.HAlign.xlHAlignRight
        # Kasvatan rivinumeroa; shape[0] antaa df1:n rivimäärän
        rivi = rivi + df1.shape[0] + 2
    ws1.range(1, 1).value = \
        'Frekvenssitaulukot kategorisille ja mielipideasteikollisille muuttujille'
    ws1.range(1, 1).font.bold = True


#Näytetään Excel
app.visible = True

Lisätietoa Pythonin käytöstä data-analytiikassa https://tilastoapu.wordpress.com/python/